In [53]:
#Download the content
import requests
#Parse 
from bs4 import BeautifulSoup
#Catch the content
from lxml import etree
#Extrair números e símbolos
import regex as re
#Biblioteca spacy
import spacy

In [54]:
RE_NORMALIZE_BLANKS = re.compile(r'[\s]+')
url = "http://www.planalto.gov.br/ccivil_03/constituicao/ConstituicaoCompilado.htm"
REQUEST_HEADER = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

#Clean the text
def replace_all_problems(text):
    fim = text.replace("\r\n", " ")
    return fim

#Normalize the text
def normalize_text(s):

    return RE_NORMALIZE_BLANKS.sub(' ', s).strip()

#Request the html
def download_html(url):
    r = requests.get(url, headers=REQUEST_HEADER)

    return r.text

#Split the html in the end of art 250
def remove_pos_normas_centrais(html):

    return html.split("Brasília, 5 de outubro de 1988.")[0]

#Parse the soup
def parse_soup(html):
    soup = BeautifulSoup(html, 'html.parser')
    
    return soup

#Extract the preambulo
def extract_preambulo(soup):
    dom = etree.HTML(str(soup))
    content = replace_all_problems((dom.xpath('/html/body/p[3]/font/small')[0].text))
    dictionary = {"preambulo": str(content)}
    return dictionary

def create_title(soup):
    find = soup.find_all("p", face="Arial")
    return find

html = download_html(url)
html_preambulo_e_normas_centrais = remove_pos_normas_centrais(html)
soup = parse_soup(html_preambulo_e_normas_centrais)

cf = ' '.join(p.text for p in soup.find_all('p'))
cf = normalize_text(cf)
cf = cf.strip()
cf = re.findall('TÍTULO\s+[IXV]+.*?(?=TÍTULO)', cf)

In [55]:
cf[0]

'TÍTULO I Dos Princípios Fundamentais Art. 1º A República Federativa do Brasil, formada pela união indissolúvel dos Estados e Municípios e do Distrito Federal, constitui-se em Estado Democrático de Direito e tem como fundamentos: I - a soberania; II - a cidadania III - a dignidade da pessoa humana; IV - os valores sociais do trabalho e da livre iniciativa; (Vide Lei nº 13.874, de 2019) V - o pluralismo político. Parágrafo único. Todo o poder emana do povo, que o exerce por meio de representantes eleitos ou diretamente, nos termos desta Constituição. Art. 2º São Poderes da União, independentes e harmônicos entre si, o Legislativo, o Executivo e o Judiciário. Art. 3º Constituem objetivos fundamentais da República Federativa do Brasil: I - construir uma sociedade livre, justa e solidária; II - garantir o desenvolvimento nacional; III - erradicar a pobreza e a marginalização e reduzir as desigualdades sociais e regionais; IV - promover o bem de todos, sem preconceitos de origem, raça, sexo

In [56]:
FIND_GROUP_TITLE = re.compile('TÍTULO\s+[IXV]+.*?(?=TÍTULO)')
FIND_TITLE = re.compile('(?P<titulo>TÍTULO)\s+(?P<titulo_numero>[IXV]+)\s+(?P<titulo_descricao>.*?(?=Art))')
FIND_CHAPTER = re.compile('CAPÍTULO')

In [59]:
titulo = {}
titulo['titles'] = [
    {
        'titulo': m.group('titulo'),
        'titulo_identificador': m.group('titulo_numero'),
        'titulo_descricao': m.group('titulo_descricao').strip(),
        'chapter': {
            'capitulo': m.group('capitulo') if FIND_CHAPTER.match(cf[0]) else '0',
            'capitulo_descricao': m.group('capitulo_descricao') if FIND_CHAPTER.match(cf[0]) else 'CAPÍTULO ÚNICO'
            'artigo': {
                'artigo_identificador': 
                
            }
                
        }
    } for m in FIND_TITLE.finditer(cf[0])
]
titulo

{'titles': [{'titulo': 'TÍTULO',
   'titulo_identificador': 'I',
   'titulo_descricao': 'Dos Princípios Fundamentais',
   'chapter': {'capitulo': '0', 'capitulo_descricao': 'CAPÍTULO ÚNICO'}}]}